In [81]:
import sys
assert sys.version_info >= (3,5)
import sklearn
assert sklearn.__version__ >= "0.20"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.plotting import scatter_matrix
import time
import cv2
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB 
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import cross_val_score

In [82]:
binary_class_x_train=pd.read_csv('./Traffic_Sign/Binary_Classification/x_train_bin.csv')
binary_class_y_train=pd.read_csv('./Traffic_Sign/Binary_Classification/y_train_bin.csv')
binary_class_x_test=pd.read_csv('./Traffic_Sign/Binary_Classification/x_test_bin.csv')
binary_class_y_test=pd.read_csv('./Traffic_Sign/Binary_Classification/y_test_bin.csv')

complete_x_train=pd.read_csv('./Traffic_Sign/CompleteDataSet/x_train_all.csv')
complete_y_train=pd.read_csv('./Traffic_Sign/CompleteDataSet/y_train_all.csv')
complete_x_test=pd.read_csv('./Traffic_Sign/CompleteDataSet/x_test_all.csv')
complete_y_test=pd.read_csv('./Traffic_Sign/CompleteDataSet/y_test_all.csv')

onevsall_y_train_0=pd.read_csv('./Traffic_Sign/OnevrsAll/0_vrs_all/y_train_0.csv')
onevsall_y_train_1=pd.read_csv('./Traffic_Sign/OnevrsAll/1_vrs_all/y_train_1.csv')
onevsall_y_train_2=pd.read_csv('./Traffic_Sign/OnevrsAll/2_vrs_all/y_train_2.csv')
onevsall_y_train_3=pd.read_csv('./Traffic_Sign/OnevrsAll/3_vrs_all/y_train_3.csv')
onevsall_y_train_4=pd.read_csv('./Traffic_Sign/OnevrsAll/4_vrs_all/y_train_4.csv')
onevsall_y_train_5=pd.read_csv('./Traffic_Sign/OnevrsAll/5_vrs_all/y_train_5.csv')
onevsall_y_train_6=pd.read_csv('./Traffic_Sign/OnevrsAll/6_vrs_all/y_train_6.csv')
onevsall_y_train_7=pd.read_csv('./Traffic_Sign/OnevrsAll/7_vrs_all/y_train_7.csv')
onevsall_y_train_8=pd.read_csv('./Traffic_Sign/OnevrsAll/8_vrs_all/y_train_8.csv')
onevsall_y_train_9=pd.read_csv('./Traffic_Sign/OnevrsAll/9_vrs_all/y_train_9.csv')

# Specs assumes 0 is true and 1 is false. So we have to reverse that

def inverse_num(i):
    if i == 1: return 0
    else: return 1

onevsall_y_train_0 = [[inverse_num(i)] for i in onevsall_y_train_0.copy().to_numpy()]
onevsall_y_train_1 = [[inverse_num(i)] for i in onevsall_y_train_1.copy().to_numpy()]
onevsall_y_train_2 = [[inverse_num(i)] for i in onevsall_y_train_2.copy().to_numpy()]
onevsall_y_train_3 = [[inverse_num(i)] for i in onevsall_y_train_3.copy().to_numpy()]
onevsall_y_train_4 = [[inverse_num(i)] for i in onevsall_y_train_4.copy().to_numpy()]
onevsall_y_train_5 = [[inverse_num(i)] for i in onevsall_y_train_5.copy().to_numpy()]
onevsall_y_train_6 = [[inverse_num(i)] for i in onevsall_y_train_6.copy().to_numpy()]
onevsall_y_train_7 = [[inverse_num(i)] for i in onevsall_y_train_7.copy().to_numpy()]
onevsall_y_train_8 = [[inverse_num(i)] for i in onevsall_y_train_8.copy().to_numpy()]
onevsall_y_train_9 = [[inverse_num(i)] for i in onevsall_y_train_9.copy().to_numpy()]

complete_x_train = complete_x_train.astype('float') / 255
complete_x_test = complete_x_test.astype('float') / 255

In [83]:
def img_denoiser(arr):
    pixel_array = arr
    reshaped_array = pixel_array.reshape((48, 48))
    normalized_array = np.uint8(reshaped_array * 255)
    if normalized_array.ndim != 2:
        raise ValueError("The image should have 2 dimensions")
    thresholded_image = cv2.adaptiveThreshold(normalized_array, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return thresholded_image

In [84]:
complete_x_train = np.apply_along_axis(
    func1d=lambda img: img_denoiser(img),
    axis=1, arr=complete_x_train).reshape(-1, 48*48)

complete_x_test = np.apply_along_axis(
    func1d=lambda img: img_denoiser(img),
    axis=1, arr=complete_x_test).reshape(-1, 48*48)

complete_x_train = complete_x_train.astype('float') / 255
complete_x_test = complete_x_test.astype('float') / 255

In [85]:
complete_x_train = pd.DataFrame(complete_x_train)
complete_x_test = pd.DataFrame(complete_x_test)

In [86]:
final_dataset = complete_x_train.copy()
final_dataset['class']=complete_y_train.copy()

In [87]:
def init_train_classifier(classifier, dataset_train_x, dataset_train_y):
    clf = None
    if classifier == "gaussian":
        clf = GaussianNB()
    elif classifier == "multinomial":
        clf = MultinomialNB()
    elif classifier == "complement":
        clf = ComplementNB()
    elif classifier == "categorical":
        clf = CategoricalNB(min_categories=10)
    else:
        raise ValueError(f"Unknown classifier: {classifier}")
    clf.fit(dataset_train_x, dataset_train_y)
    return clf

def run_naivebayes(classifier, dataset_train_x, dataset_train_y, dataset_test_x, dataset_test_y):
    clf = init_train_classifier(classifier, dataset_train_x, dataset_train_y)
    complete_y_pred = clf.predict(dataset_test_x)
    ret_arr = [
        accuracy_score(dataset_test_y, complete_y_pred), 
        precision_score(dataset_test_y, complete_y_pred, average='weighted'), 
        recall_score(dataset_test_y, complete_y_pred, average='weighted'),
        f1_score(dataset_test_y, complete_y_pred, average='weighted'),
        classification_report(dataset_test_y, complete_y_pred),
        confusion_matrix(dataset_test_y, complete_y_pred)
    ]
    return ret_arr

In [88]:
temp_dataset = final_dataset.groupby('class', group_keys=False)
avg_val = int(temp_dataset['class'].value_counts().mean())

balanced_dataset = pd.DataFrame()

for i in range(0,10):
    if (len(final_dataset[final_dataset['class']==i])<=avg_val):
        balanced_dataset = pd.concat([balanced_dataset, final_dataset[final_dataset['class']==i].sample(avg_val, replace=True)])
    else:
        balanced_dataset = pd.concat([balanced_dataset, final_dataset[final_dataset['class']==i].sample(avg_val)])

balanced_dataset_x = balanced_dataset.iloc[:,:2304]
balanced_dataset_y = balanced_dataset.iloc[:,2304:]


In [89]:
class_arr_data = [
                onevsall_y_train_0.copy(),
                onevsall_y_train_1.copy(),
                onevsall_y_train_2.copy(),
                onevsall_y_train_3.copy(),
                onevsall_y_train_4.copy(),
                onevsall_y_train_5.copy(),
                onevsall_y_train_6.copy(),
                onevsall_y_train_7.copy(),
                onevsall_y_train_8.copy(),
                onevsall_y_train_9.copy()
            ]

x_data = complete_x_train.copy()

def getTopFeatureMethod1(x,dataset,arr):
    return_arr = []
    for i in arr:
        bestfeatures = SelectKBest(score_func=chi2, k=x//len(arr))
        fit = bestfeatures.fit(dataset,i)
        dfscores = pd.DataFrame(fit.scores_)
        dfcolumns = pd.DataFrame(dataset.columns)
        featureScores = pd.concat([dfcolumns,dfscores],axis=1)
        featureScores.columns = ['Specs','Score']
        return_arr = np.unique(np.concatenate((return_arr,featureScores.nlargest(x//len(arr),'Score')['Specs']),0))
    return return_arr

def Remove_Outlier_Indices(df):
    Q1 = df.quantile(0.2)
    Q3 = df.quantile(0.8)
    IQR = Q3 - Q1
    trueList = ~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)))
    return trueList

def getTopFeatureMethod2(x,dataset,arr):
    features_array = [[]]*10
    return_arr = [[]]*3
    for i in range (0,len(arr)):
        bestfeatures = SelectKBest(score_func=chi2, k=x//len(arr))
        fit = bestfeatures.fit(dataset,arr[i])
        dfscores = pd.DataFrame(fit.scores_)
        dfcolumns = pd.DataFrame(dataset.columns)
        featureScores = pd.concat([dfcolumns,dfscores],axis=1)
        featureScores.columns = ['Specs','Score']
        features_array[i]=featureScores.nlargest(x//len(arr),'Score')['Specs']
    for i in range (0,10):
        return_arr[0] = np.unique(np.concatenate((return_arr[0], features_array[i][0:x//len(arr)]),0))
        return_arr[1] = np.unique(np.concatenate((return_arr[1], features_array[i][0:x//(len(arr)*2)]),0))
        return_arr[2] = np.unique(np.concatenate((return_arr[2], features_array[i][0:x//(len(arr)*4)]),0))
    return return_arr

In [90]:
temp1 = getTopFeatureMethod1(400,x_data,class_arr_data)
temp2 = getTopFeatureMethod1(200,x_data,class_arr_data)
temp3 = getTopFeatureMethod1(125,x_data,class_arr_data)

temp = getTopFeatureMethod2(700,x_data,class_arr_data)

In [91]:
non_outlier1 = Remove_Outlier_Indices(balanced_dataset_x)
non_outlier2 = Remove_Outlier_Indices(complete_x_test)

In [92]:
balanced_dataset_x=balanced_dataset_x[non_outlier1]
balanced_dataset_x=balanced_dataset_x.fillna(0)

complete_x_test=complete_x_test[non_outlier2]
complete_x_test=complete_x_test.fillna(0)

In [93]:
nb=MultinomialNB()
arr=temp3

In [94]:
nb.fit(balanced_dataset_x[arr],balanced_dataset_y)

y_pred_nb=nb.predict(complete_x_test[arr])

print(accuracy_score(y_pred_nb,complete_y_test))
print(classification_report(y_pred_nb,complete_y_test))

0.6650485436893204
              precision    recall  f1-score   support

           0       0.73      0.34      0.46       131
           1       0.61      0.77      0.68       568
           2       0.72      0.75      0.74       725
           3       0.63      0.63      0.63       446
           4       0.66      0.80      0.72       541
           5       0.93      0.39      0.55       143
           6       0.69      0.38      0.49       164
           7       0.55      0.37      0.44        89
           8       0.67      0.76      0.71       133
           9       0.70      0.42      0.53       150

    accuracy                           0.67      3090
   macro avg       0.69      0.56      0.59      3090
weighted avg       0.68      0.67      0.65      3090



In [95]:
datasets = [
    [(balanced_dataset_x[temp1], complete_x_test[temp1]), (balanced_dataset_x[temp2], complete_x_test[temp2]), (balanced_dataset_x[temp3], complete_x_test[temp3])],
    [(balanced_dataset_x[temp[0]], complete_x_test[temp[0]]), (balanced_dataset_x[temp[1]], complete_x_test[temp[1]]), (balanced_dataset_x[temp[2]], complete_x_test[temp[2]])]
]

bayes = ["gaussian", "multinomial", "complement"]

for i in range (0,len(datasets)):
    print("*"*100)
    print("Using Method ",i+1)
    print("*"*100)
    for j in range (0,len(datasets[i])):
        for k in range (0,len(bayes)):
            temp = run_naivebayes(bayes[k], datasets[i][j][0], balanced_dataset_y, datasets[i][j][1], complete_y_test)
            print("Dataset",i+1,"\tMethod",j+1,"\t",bayes[k].capitalize(),"Naive Bayes","    \tAccuracy :",temp[0],"    \tPrecision :",temp[1],"    \tRecall Score :",temp[2],"    \tF1 Score :",temp[3])

****************************************************************************************************
Using Method  1
****************************************************************************************************


Dataset 1 	Method 1 	 Gaussian Naive Bayes     	Accuracy : 0.7378640776699029     	Precision : 0.7702593929524392     	Recall Score : 0.7378640776699029     	F1 Score : 0.7412212796521143
Dataset 1 	Method 1 	 Multinomial Naive Bayes     	Accuracy : 0.6967637540453074     	Precision : 0.7319449026969754     	Recall Score : 0.6967637540453074     	F1 Score : 0.7047481387379192
Dataset 1 	Method 1 	 Complement Naive Bayes     	Accuracy : 0.596116504854369     	Precision : 0.7234693698510563     	Recall Score : 0.596116504854369     	F1 Score : 0.6353691717840814
Dataset 1 	Method 2 	 Gaussian Naive Bayes     	Accuracy : 0.7355987055016181     	Precision : 0.7626536446731632     	Recall Score : 0.7355987055016181     	F1 Score : 0.7399725291700247
Dataset 1 	Method 2 	 Multinomial Naive Bayes     	Accuracy : 0.7025889967637541     	Precision : 0.7320815665127461     	Recall Score : 0.7025889967637541     	F1 Score : 0.7088916896929455
Dataset 1 	Method 2 	 Complement Naive Bayes     	Accu